# Multi-Modal Fine-Tuning


## Install and import libraries

In [1]:
!pip install matplotlib -q -U
!pip install datasets -q -U
!pip install -q bitsandbytes sentencepiece  accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install hf_transfer -q -U
!pip install pickleshare -q

In [2]:
# Allows for faster downloads
%env HF_HUB_ENABLE_HF_TRANSFER=1

env: HF_HUB_ENABLE_HF_TRANSFER=1


In [3]:
import os

if not os.path.isdir('LLaVA'):
    !git clone https://github.com/haotian-liu/LLaVA.git
else:
    print("LLaVA already exists. Skipping clone.")

Klone nach 'LLaVA'...
remote: Enumerating objects: 2297, done.
remote: Total 2297 (delta 0), reused 0 (delta 0), pack-reused 2297 (from 1)
Empfange Objekte: 100% (2297/2297), 13.71 MiB | 12.02 MiB/s, fertig.
Löse Unterschiede auf: 100% (1405/1405), fertig.


In [9]:
import re

# Define the path to the builder.py file
file_path = '/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/fine-tuning/LLaVa_1.6_7B/LLaVA/llava/model/builder.py'

# Read the content of the file
with open(file_path, 'r') as file:
    content = file.read()

# Regular expression to find the block between 'vision_tower = model.get_vision_tower()' and 'vision_tower.image_processor'
pattern_block = (
    r'(vision_tower = model.get_vision_tower\(\)\n)'
    r'.*?' # Non-greedy match for any characters
    r'(image_processor = vision_tower.image_processor)'
)

replacement_block = (
    r'\1' # Keep the first line unchanged
    '       if not vision_tower.is_loaded:\n'
    '           print(\'vision_tower is not loades so loading now\')\n'
    '           vision_tower.load_model(device_map=device_map)\n'
    '           vision_tower.to(device=device_map, dtype=torch.bfloat16)\n'
    '       else:\n'
    '           print(\'vision_tower is already loaded\')\n'
    r'      \2' # Keep the last line unchanged
)

# Replace the specific block
content = re.sub(pattern_block, replacement_block, content, flags=re.DOTALL)

# Write the content back to the file
with open(file_path, 'w') as file:
    file.write(content)

print('File modified successfully.')

File modified successfully.


In [10]:
import re

# Define the path to the builder.py file
file_path = '/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/fine-tuning/LLaVa_1.6_7B/LLaVA/llava/model/builder.py'

# Read the content of the file
with open(file_path, 'r') as file:
    content = file.read()

# Regular expression to find 'float16' and replace it with 'bfloat16'
pattern = r'(?<!b)float16'

# CHeck if there are any matches
if re.search(pattern, content):
    # Replace all matches
    modified_content = re.sub(pattern, 'bfloat16', content)

    # Write the content back to the file
    with open(file_path, 'w') as file:
        file.write(content)

    print('File modified successfully.')
else:
    print('No modification needed.')

No modification needed.


In [4]:
%cd LLaVA

/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/fine-tuning/LLaVa_1.6_7B/LLaVA


In [7]:
# Takes quite a while to run
!pip install -e . -q

In [11]:
!pip install protobuf -q -U
!pip install --upgrade Pillow -q
!pip install -e '.[train]' -q
!pip install flash-attn --no-build-isolation -q

ERROR: unknown command "installl" - maybe you meant "install"
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/jw/9kbqqtt11wj27s6z8wmgk_yw0000gp/T/pip-install-2jbcb61h/deepspeed_b0e5288316eb4ed2be57c7e5651e6737/setup.py", line 38, in <module>
          from op_builder.all_ops import ALL_OPS
        File "/private/var/folders/jw/9kbqqtt11wj27s6z8wmgk_yw0000gp/T/pip-install-2jbcb61h/deepspeed_b0e5288316eb4ed2be57c7e5651e6737/op_builder/all_ops.py", line 29, in <module>
          builder = get_accelerator().create_op_builder(member_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/private/var/folders/jw/9kbqqtt11wj27s6z8wmgk_yw0000gp/T/pip-install-2jbcb61h/deepspeed_b0e5288316eb4ed

## Load the Model

In [2]:
import torch
from datasets import load_dataset
import transformers
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

In [3]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_path = 'liuhaotian/llava-v1.6-mistral-7b' # needs about 100 GB of VRAM

model_name = get_model_name_from_path(model_path)

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=model_name,
    cache_dir='',
    use_flash_attn=True,
)

/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/.conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

KeyboardInterrupt: 

## Optional examination

In [ ]:
import torch

print('modules not torch.bfloat16:')
for name, module in model.named_modules():
    if hasattr(module, 'parameters')) and list(module.parameters()):
        # Check if the module has float16 parameters
        if any(param.dtype != torch.bfloat16 for param in module.parameters()):
            print(f'{name}: {next(module.parameters()).dtype}')
        else:
            pass

## Inference

In [ ]:
# Method to test the model's inference
import torch
import re
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.transforms.functional import to_pil_image, to_tensor
from PIL import Image
import requests
from io import BytesIO

from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)

# Common function to create prompts
def create_prompt(query, model, model_name=model_name, caption=None):
    image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
    if IMAGE_PLACEHOLDER in query:
        if model.config.mm_use_im_start_end:
            query = query.replace(IMAGE_PLACEHOLDER, image_token_se, query)
        else:
            query = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, query)
    else:
        if model.config.mm_use_im_start_end:
            query = image_token_se + '\n' + query
        else:
            query = DEFAULT_IMAGE_TOKEN + '\n' + query

        conv_mode = infer_conv_mode(model_name)
        conv = conv_templates[conv_mode].copy()
        conv.append_message(conv.roles[0], query)
        if caption is not None:
            conv.append_message(conv.roles[1], caption)
        else:
            conv.append_message(conv.roles[1], None)
        return conv.get_prompt()
    
# Common function to infer conversation mode
def infer_conv_mode(model_name):
    if 'llama-2' in model_name.lower():
        return ' llava_llama_2'
    else:
        return 'mistral_instruct'
        
# Common function to process images
def process_and_prepare_images(image_files, image_processor, model, device):
    images = [load_image(image_file) for image_file in image_files]
    image_tensor = process_images(
        images, 
        image_processor, 
        model.config
    ).to(
        device,
        dtype=torch.bfloat16
    )
    image_sizes = [image.size for image in images]
    return image_tensor, image_sizes

In [ ]:
import torch
import re

def load_image(image_input):
    # Check if the input is a string (path/url)
    if isinstance(image_input, str):
        if image_input.startswith('http') or image_input.startswith('https'):
            response = requests.get(image_input)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        else:
            image = Image.open(image_input).convert('RGB')
    elif isinstance(image_input, Image.Image):
        # Input is already an image
        image = image_input
    else:
        raise ValueError('Invalid input. Please provide a valid input type.')
    return image

def eval_model(tokenizer, model, image_processor, context_len, image_file, query. model_name=model_name, sep=',', temperature=1.0, num_beams=1, max_new_tokens=512):
    # Model
    disable_torch_init()

    # Create prompt using the common function
    prompt = create_prompt('Describe the image.',model, model_name)

    print(f'Prompt: {prompt}')

    # Process images using the common function
    if isinstance(image_file, list):
        image_tensor, image_sizes = process_and_prepare_images(image_file, image_processor, model, model.device)
    elif isinstance(image_file, str):
        image_tensors, image_sizes = process_and_prepare_images([image_file], image_processor, model, model.device)
    else:
        # If image_files is not a list or a string, it's likely an Image Object
        images = [image_file]
        image_tensors, image_sizes = process_and_prepare_images(images, image_processor, model, model.device)

    # Tokenize the prompt using the custom tokenizer_image_token function
    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt')
        .unsqueeze(0)
        .to(model.device)
    )

    with torch.inference_mode():
        output_ids = mode.generate(
            input_ids,
            images=image_tensor,
            image_sizes=image_sizes,
            do_sample=temperature != 1.0,
            temperature=temperature,
            num_beams=num_beams,
            max_new_tokens=max_new_tokens,
            use_cache=True,
        )
        
    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=False)
    print(outputs)


In [ ]:
import requests
from PIL import Image
from io import BytesIO

# Raw image URL from GitHub
image_url = 'https://github.com/fuerstfabian/Fine-tuned-LLaVA-Vision-and-Language/blob/2ba3fa1bdec75ddb685738c8afd8560fd9d9d30f/data_prep/data/figure_9647.jpg'

# Download image and open it with PIL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

import matplotlib.pyplot as plt
plt.imshow(image)
plt.axis('off')
plt.show()

# Pass processed image to the eval_model function
eval_model(
    tokenizer,
    model,
    image_processor,
    context_len, 
    image, 
    'What do you see in the image?', 
    model_name=model_name
)

## Fine-Tuning Dataset

Vorbereiten des Datasets, welches für das Fine-Tuning verwendet wird

In [ ]:
from torch.utils.rnn import pad_sequence

def tokenize_and_create_labels(example_batch, image_processor, tokenizer):
    pad_token_id = tokenizer.pad_token_id
    image_files = example_batch['image']

    images_tensor, image_sizes = process_and_prepare_images(image_files, image_processor, model, model.device)

    query = 'What do you see in the image?'

    # Tokenize the conversation without the captions to determine which tokens are used
    tokenized_conversations_without_caption = [
        tokenizer_image_token(create_prompt(query, model, model_name, None))
        for _ in example_batch['caption']
    ]

    # Tokenize the full conversation
    tokenized_conversations_with_caption = [
        tokenizer_image_token(create_prompt(query, model, model_name, caption))
        for caption in example_batch['caption']
    ]

    # Pad the tokenized conversations to the same length
    input_ids = pad_sequence([tcwc.squeeze(0) for tcwc in tokenized_conversations_with_caption], batch_first=True, padding_value=pad_token_id)

    # Create attention_mask (1 for real tokens and 0 for padding tokens)
    attention_mask = (input_ids != pad_token_id).long().to(device)

    # Create the labels tensor which is a copy of input_ids but with ignore_index for padding tokens
    labels = torch.full_like(input_ids, fill_value=ignore_index)
    for i, tcwc in enumerate(tokenized_conversations_without_caption):
        # Set ignore_index for the tokens corresponding to the conversation
        input_id_without_caption = tcwc.squeeze(0)
        labels[i, len(input_id_without_caption):] = input_ids[i, len(input_id_without_caption):]

    inputs = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'images': images_tensor,
        'image_sizes': image_sizes,
    }

    return inputs

# Make sure to define the function outside of the lambda to ensure it's picklable
def transform_batch(batch):
    return tokenize_and_create_labels(batch, image_processor, tokenizer, model)

# load and prepare the dataset
ds = load_dataset('fuerstfabian/cat_figures')

train_ds = ds['train']
eval_ds = ds['test']

# Apply the transformation function to the dataset
train_ds.set_transform(transform_batch)
eval_ds.set_transform(transform_batch)

## LoRA

Nachdem erstellen der Config des Low-rank Adapters (LoRA) können wir das PeftModel mit der get_peft_model Funktion laden

In [ ]:
print(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        'q_proj', 'k_proj', 'v_proj',
        'mm_projector', # for mistral, train instead 'mm_projector'
        'down_proj', 'gate_proj'
    ],
    lora_dropout=0.05,
    bias='none',
)
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

## Pre-Training Evaluation